# Make Submission: 自作yolov8でロゴ部分のみ抽出 version (Nishika 2nd solutionのconvnext，AugLy部分を排除して利用)
ref: https://github.com/anyai-28/nishika_jpo_2nd_solution?tab=readme-ov-file

手順
1. convnextのTrainingが先に終わった。そして、train_swinの実行が長すぎる
2. train_swinの実行を待っていられない
3. convnextだけでsubmissionを作るコードを書く

1.train_conv.py ~->~ ~2.train_swin.py~ -> 3.submission.py


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## download directory structure and csv files

In [2]:
%cd /content/

/content


In [3]:
# it is the Nishika trademark competition datasets (https://competition.nishika.com/competitions/patent/data)
!pip install gdown
!gdown --folder https://drive.google.com/drive/folders/1KUpQ6ZyIawgALEsyzWBt4fr7s8TT62vM?usp=drive_link

Retrieving folder list
Processing file 1-ObndOWm0jpljV7PTps6mOqtW-2lNM77 cite.csv
Processing file 1-OLYdb0NPMHgtyCmQ93jUGp8CgzNN03a cpt-jpo-tutorial.zip
Processing file 1vYJZ-s8ppZtT66iTyJh1vSbmHEtGThCy my_train_w_group_label.csv
Processing file 1-kL8bZKY3ZQeRIIBJSd0NNq55NscIxUc sample_submission.csv
Processing file 1-cLynCOTU1QMuRSphsiOhlgIC45espef test.csv
Processing file 1-Z_rzDGZqZ-My2BOz0aUv5tjpnuLKHE6 train.csv
Retrieving folder list completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1-ObndOWm0jpljV7PTps6mOqtW-2lNM77
To: /content/common/cite.csv
100% 29.6M/29.6M [00:00<00:00, 83.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-OLYdb0NPMHgtyCmQ93jUGp8CgzNN03a
To: /content/common/cpt-jpo-tutorial.zip
100% 1.02M/1.02M [00:00<00:00, 171MB/s]
Downloading...
From: https://drive.google.com/uc?id=1vYJZ-s8ppZtT66iTyJh1vSbmHEtGThCy
To: /content/common/my_train_w_group_label.csv
100% 196k/196k [00:00<

In [4]:
!cp /content/common/cpt-jpo-tutorial.zip /content
!unzip /content/cpt-jpo-tutorial.zip

Archive:  /content/cpt-jpo-tutorial.zip
  inflating: requirements.txt        
  inflating: __MACOSX/._requirements.txt  
   creating: data/
  inflating: data/.DS_Store          
  inflating: __MACOSX/data/._.DS_Store  
   creating: data/input/
   creating: data/output/
  inflating: data/input/.gitkeep     
  inflating: data/output/.gitkeep    
   creating: models/
  inflating: models/.gitkeep         
   creating: notebooks/
  inflating: notebooks/.DS_Store     
  inflating: __MACOSX/notebooks/._.DS_Store  
  inflating: notebooks/tutorial_check_train_data.ipynb  
  inflating: __MACOSX/notebooks/._tutorial_check_train_data.ipynb  
   creating: scripts/
  inflating: scripts/03_tutorial_infer.py  
  inflating: __MACOSX/scripts/._03_tutorial_infer.py  
  inflating: scripts/01_tutorial_preprocess.py  
  inflating: __MACOSX/scripts/._01_tutorial_preprocess.py  
  inflating: scripts/.DS_Store       
  inflating: __MACOSX/scripts/._.DS_Store  
  inflating: scripts/02_tutorial_train.py  
  infl

In [5]:
!cp -r /content/common/* /content/data/input

## download yolov8 Croped input image

crop_cite_images is **too large** cannot download gdown
- you can use the way
ref: https://yuiga.dev/blog/posts/%E5%B7%A8%E5%A4%A7%E3%83%87%E3%83%BC%E3%82%BFgdown%E3%81%AEaccess_denied%E5%95%8F%E9%A1%8C%E3%81%AF%E8%AB%A6%E3%82%81%E3%82%88%E3%81%86/


>解決策
>1. 当該URLからファイルIDを抜き出す (URLのどこがidなのかは適当にググってください)
>2. OAuth 2.0 Playgroundにアクセス
>3. “Drive API v3”を探してクリック→ 「 https://www.googleapis.com/auth/drive.>readonly 」と書いてある所をクリック
>4. “exchange authorization code for tokens"をクリックして"Access Token"を取得
>5. curl -H "Authorization: Bearer <access-token>" https://www.googleapis.com/drive/v3/files/<file-id>?alt=media -o <output-path> を実行
>6. ```<access-token>, <file-id>, <output-path>```はご自身に適切なものを入れてください

- or you can dowonload it (https://drive.google.com/drive/folders/1-1fqMU3DXTR-IAR7mRS-biMZPD6bSrE0?usp=drive_link)
and upload your colab.


In [6]:
# FILE_ID='1-ASA9GDzzk76xplGxsIRZzjqJwdNwlho'
# FILE_NAME='crop_cite_images.zip'

# yolov8 crop_cite_images https://drive.google.com/file/d/1-ASA9GDzzk76xplGxsIRZzjqJwdNwlho/view?usp=drive_link
# !curl -H "Authorization: Bearer <access token>" https://www.googleapis.com/drive/v3/files/1-ASA9GDzzk76xplGxsIRZzjqJwdNwlho?alt=media -o crop_cite_images.zip
# !curl -H "Authorization: Bearer " https://www.googleapis.com/drive/v3/files/1-ASA9GDzzk76xplGxsIRZzjqJwdNwlho?alt=media -o crop_cite_images.zip
!curl -H "Authorization: Bearer " https://www.googleapis.com/drive/v3/files/1-ASA9GDzzk76xplGxsIRZzjqJwdNwlho?alt=media -o crop_cite_images.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 37.3G  100 37.3G    0     0  62.2M      0  0:10:14  0:10:14 --:--:-- 46.3M


In [7]:
# !curl -H "Authorization: Bearer <acccess token>" https://www.googleapis.com/drive/v3/files/1-7wwn4hiKXZ3hpU1j7yyKlO7mlv43XbV?alt=media -o crop_test_images.zip
# !curl -H "Authorization: Bearer <acccess token>" https://www.googleapis.com/drive/v3/files/1DLVYkwBHUu3y7r5SKZEihfS7jHq6qApq?alt=media -o crop_train_apply_images.zip
# !curl -H "Authorization: Bearer <acccess token>" https://www.googleapis.com/drive/v3/files/1-28QPaUHLJZlbWKTUK4RMJvV8TenVlj8?alt=media -o crop_train_cite_images.zip


# yolov8 crop_test_images https://drive.google.com/file/d/1-7wwn4hiKXZ3hpU1j7yyKlO7mlv43XbV/view?usp=drive_link
!curl -H "Authorization: Bearer " https://www.googleapis.com/drive/v3/files/1-7wwn4hiKXZ3hpU1j7yyKlO7mlv43XbV?alt=media -o crop_test_images.zip
# yolov8 crop_train_apply_images https://drive.google.com/file/d/1DLVYkwBHUu3y7r5SKZEihfS7jHq6qApq/view?usp=drive_link
!curl -H "Authorization: Bearer " https://www.googleapis.com/drive/v3/files/1DLVYkwBHUu3y7r5SKZEihfS7jHq6qApq?alt=media -o crop_train_apply_images.zip
# yolov8 crop_train_cite_images https://drive.google.com/file/d/1-28QPaUHLJZlbWKTUK4RMJvV8TenVlj8/view?usp=drive_link
!curl -H "Authorization: Bearer " https://www.googleapis.com/drive/v3/files/1-28QPaUHLJZlbWKTUK4RMJvV8TenVlj8?alt=media -o crop_train_cite_images.zip

# !gdown https://drive.google.com/uc?id=1-7wwn4hiKXZ3hpU1j7yyKlO7mlv43XbV  # yolov8 crop_test_images https://drive.google.com/file/d/1-7wwn4hiKXZ3hpU1j7yyKlO7mlv43XbV/view?usp=drive_link
# !gdown https://drive.google.com/uc?id=1DLVYkwBHUu3y7r5SKZEihfS7jHq6qApq  # yolov8 crop_train_apply_images https://drive.google.com/file/d/1DLVYkwBHUu3y7r5SKZEihfS7jHq6qApq/view?usp=drive_link
# !gdown https://drive.google.com/uc?id=1-28QPaUHLJZlbWKTUK4RMJvV8TenVlj8  # yolov8 crop_train_cite_images https://drive.google.com/file/d/1-28QPaUHLJZlbWKTUK4RMJvV8TenVlj8/view?usp=drive_link

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 71.1M  100 71.1M    0     0  4245k      0  0:00:17  0:00:17 --:--:-- 18.0M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  111M  100  111M    0     0  34.6M      0  0:00:03  0:00:03 --:--:-- 34.6M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 95.6M  100 95.6M    0     0  3924k      0  0:00:24  0:00:24 --:--:-- 20.2M


In [ ]:
!unzip /content/crop_cite_images.zip -d /content/data/input/
!unzip /content/crop_train_apply_images.zip -d /content/data/input/
!unzip /content/crop_train_cite_images.zip -d /content/data/input/
!unzip /content/crop_test_images.zip -d /content/data/input/

In [9]:
!rm /content/crop_cite_images.zip
!rm /content/crop_train_apply_images.zip
!rm /content/crop_train_cite_images.zip
!rm /content/crop_test_images.zip

## jsonの通りのディレクトリ構成にしておく
ref: https://github.com/anyai-28/nishika_jpo_2nd_solution

In [10]:
%cd /content/

/content


In [11]:
"""
{
    "RAW_DATA_DIR": "./data/",
    "PROCESSED_DATA_DIR": "./data/processed/",
    "MODEL_CHECKPOINT_DIR": "./models/",
    "PRETRAINED_MODEL_CHECKPOINT_DIR": "./pretrained_models/",
    "USE_PRETRAIN": "False",
    "LOGS_DIR": "./logs/",
    "SUBMISSION_DEVICE": "cuda",
    "SUBMISSION_DIR": "./submissions/"}
"""
!mkdir ./data/processed
!mkdir ./pretrained_models/
!mkdir ./submissions/

!mkdir ./config
# !mkdir ./pretrained_models/swin
!mkdir ./pretrained_models/convnext
# !mkdir ./logs
# !mkdir ./logs/swin
# !mkdir ./logs/convnext
# !mkdir ./models/convnext

import my_train_w_group_label.csv, config.yaml and settings.json

In [12]:
!pip install gdown
!gdown https://drive.google.com/uc?id=1ZOmSdpwuiPvy8PQ0QBf2NaCJ6yh01WE2 # my preprocessed csv
!mv /content/my_train_w_group_label.csv /content/data/input/train_w_group_label.csv

Downloading...
From: https://drive.google.com/uc?id=1ZOmSdpwuiPvy8PQ0QBf2NaCJ6yh01WE2
To: /content/my_train_w_group_label.csv
100% 196k/196k [00:00<00:00, 84.8MB/s]


In [13]:
!gdown https://drive.google.com/uc?id=12F5BWiQdcnS1-FvHRMkTuLFa7lxoPUOu #config_train_convnext.yaml https://drive.google.com/file/d/12F5BWiQdcnS1-FvHRMkTuLFa7lxoPUOu/view?usp=drive_link
!mv /content/config_train_convnext.yaml /content/config

Downloading...
From: https://drive.google.com/uc?id=12F5BWiQdcnS1-FvHRMkTuLFa7lxoPUOu
To: /content/config_train_convnext.yaml
100% 6.02k/6.02k [00:00<00:00, 16.8MB/s]


In [17]:
!gdown https://drive.google.com/uc?id=12QkWUsjk_JOLT1xI5kWYifcw3nWkGTpn #settings.json https://drive.google.com/file/d/12QkWUsjk_JOLT1xI5kWYifcw3nWkGTpn/view?usp=drive_link
# !mv /content/settings.json /content/

Downloading...
From: https://drive.google.com/uc?id=12QkWUsjk_JOLT1xI5kWYifcw3nWkGTpn
To: /content/settings.json
100% 320/320 [00:00<00:00, 1.54MB/s]


In [15]:
!mv /content/data/input/crop_train_apply_images /content/data/processed/
!mv /content/data/input/crop_train_cite_images /content/data/processed/
!mv /content/data/input/crop_test_images /content/data/processed
!mv /content/data/input/crop_cite_images /content/data/processed/
!mv /content/data/input/train_w_group_label.csv /content/data/processed/
!mv /content/data/input/test.csv /content/data/
!mv /content/data/input/cite.csv /content/data/
!mv /content/data/input/sample_submission.csv /content/data/

In [16]:
!mv /content/data/processed/crop_train_apply_images /content/data/processed/crop_apply_images

import apply

In [18]:
!gdown https://drive.google.com/uc?id=19oiwap-xGKOF9u931Cacwpw2P_w-DbHJ # default apply_images #https://drive.google.com/file/d/19oiwap-xGKOF9u931Cacwpw2P_w-DbHJ/view?usp=drive_link

Downloading...
From: https://drive.google.com/uc?id=19oiwap-xGKOF9u931Cacwpw2P_w-DbHJ
To: /content/apply_images.zip
100% 176M/176M [00:03<00:00, 50.5MB/s]


In [19]:
!unzip /content/apply_images.zip -d /content/data | tail -n 1

  inflating: /content/data/apply_images/1000060501/1000060501.jpg  


Import yolov8 trained convnext model

In [20]:
!mkdir /content/tmp
%cd /content/tmp

/content/tmp


In [21]:
# ref: https://yuiga.dev/blog/posts/%E5%B7%A8%E5%A4%A7%E3%83%87%E3%83%BC%E3%82%BFgdown%E3%81%AEaccess_denied%E5%95%8F%E9%A1%8C%E3%81%AF%E8%AB%A6%E3%82%81%E3%82%88%E3%81%86/
# >解決策
# >1. 当該URLからファイルIDを抜き出す (URLのどこがidなのかは適当にググってください)
# >2. [OAuth 2.0 Playground](https://developers.google.com/oauthplayground/)にアクセス
# >3. “Drive API v3”を探してクリック→ 「 https://www.googleapis.com/auth/drive.>readonly 」と書いてある所をクリック
# >4. “exchange authorization code for tokens"をクリックして"Access Token"を取得
# >5. curl -H "Authorization: Bearer <access-token>" https://www.googleapis.com/drive/v3/files/<file-id>?alt=media -o <output-path> を実行
# >6. ```<access-token>, <file-id>, <output-path>```はご自身に適切なものを入れてください

#FILE_ID=1-3qgyIDEp6LrQCzOF3ZG_YtlWsQ9pvao/
# https://drive.google.com/file/d/1-3qgyIDEp6LrQCzOF3ZG_YtlWsQ9pvao/view?usp=drive_link

#!curl -H "Authorization: Bearer <Access token>" https://www.googleapis.com/drive/v3/files/1-3qgyIDEp6LrQCzOF3ZG_YtlWsQ9pvao?alt=media -o yolov8_ver_models.zip
!curl -H "Authorization: Bearer " https://www.googleapis.com/drive/v3/files/1-3qgyIDEp6LrQCzOF3ZG_YtlWsQ9pvao?alt=media -o yolov8_ver_models.zip


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 11.8G  100 11.8G    0     0  72.6M      0  0:02:47  0:02:47 --:--:-- 35.8M


In [22]:
!rm -rf /content/models

In [23]:
!unzip /content/tmp/yolov8_ver_models.zip -d /content

Archive:  /content/tmp/yolov8_ver_models.zip
   creating: /content/models/
   creating: /content/models/convnext/
  inflating: /content/models/convnext/train_embed_3.npy  
  inflating: /content/models/convnext/cite_embed_1.npy  
  inflating: /content/models/convnext/cite_embed_2.npy  
  inflating: /content/models/convnext/train_embed_1.npy  
  inflating: /content/models/convnext/last_fold_1.ckpt  
  inflating: /content/models/convnext/train_embed_2.npy  
  inflating: /content/models/convnext/best_val_loss_fold_1.ckpt  
  inflating: /content/models/convnext/cite_embed_3.npy  
  inflating: /content/models/convnext/cite_embed_4.npy  
  inflating: /content/models/convnext/last.ckpt  
  inflating: /content/models/convnext/best_val_loss_fold_4.ckpt  
  inflating: /content/models/convnext/train_embed_0.npy  
  inflating: /content/models/convnext/cite_mean_embed.npy  
  inflating: /content/models/convnext/best_val_loss_fold_2.ckpt  
  inflating: /content/models/convnext/last_fold_4.ckpt  
  in

In [24]:
!gdown https://drive.google.com/uc?id=1-CKOShv3dLoMPRFIzaC07SUT6N8W2NO2  #logs yolov8_ver #https://drive.google.com/file/d/1-CKOShv3dLoMPRFIzaC07SUT6N8W2NO2/view?usp=drive_link

Downloading...
From: https://drive.google.com/uc?id=1-CKOShv3dLoMPRFIzaC07SUT6N8W2NO2
To: /content/tmp/crop_yolov8_ver_logs.zip
100% 8.48k/8.48k [00:00<00:00, 31.9MB/s]


In [25]:
!unzip /content/tmp/crop_yolov8_ver_logs.zip -d /content

Archive:  /content/tmp/crop_yolov8_ver_logs.zip
   creating: /content/logs/
   creating: /content/logs/convnext/
   creating: /content/logs/convnext/fold_0/
   creating: /content/logs/convnext/fold_1/
   creating: /content/logs/convnext/fold_2/
   creating: /content/logs/convnext/fold_3/
   creating: /content/logs/convnext/fold_4/
   creating: /content/logs/convnext/fold_2/version_0/
  inflating: /content/logs/convnext/fold_2/version_0/metrics.csv  
  inflating: /content/logs/convnext/fold_2/version_0/hparams.yaml  
   creating: /content/logs/convnext/fold_0/version_0/
  inflating: /content/logs/convnext/fold_0/version_0/metrics.csv  
  inflating: /content/logs/convnext/fold_0/version_0/hparams.yaml  
   creating: /content/logs/convnext/fold_3/version_0/
  inflating: /content/logs/convnext/fold_3/version_0/metrics.csv  
  inflating: /content/logs/convnext/fold_3/version_0/hparams.yaml  
   creating: /content/logs/convnext/fold_4/version_0/
  inflating: /content/logs/convnext/fold_4/ver

In [26]:
!rm -r /content/tmp/*
%cd /content/

/content


import my-yolov8

In [33]:
!gdown https://drive.google.com/uc?id=1X2WXqyPTaE8URHi6ljQpimzM3MfcZSNB #yolov8 model https://drive.google.com/file/d/1X2WXqyPTaE8URHi6ljQpimzM3MfcZSNB/view?usp=drive_link

Downloading...
From: https://drive.google.com/uc?id=1X2WXqyPTaE8URHi6ljQpimzM3MfcZSNB
To: /content/best_v3_01.pt
100% 22.5M/22.5M [00:00<00:00, 206MB/s]


In [ ]:
!mkdir /content/my-yolov8-model/

In [35]:
!mv /content/best_v3_01.pt /content/my-yolov8-model/best.pt

## 環境構築

https://stackoverflow.com/questions/70585934/unsatisfiableerror-when-installing-cuml

In [ ]:
!nvidia-smi

Sun Dec 10 05:39:08 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## 追加

python3.10.*   
cuda11.8

In [36]:
#cuda 11.*系にダウングレード
#https://developer.nvidia.com/cuda-11-8-0-download-archive?target_os=Linux&target_arch=x86_64&Distribution=Ubuntu&target_version=22.04&target_type=deb_local

!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/cuda-ubuntu2204.pin
!mv cuda-ubuntu2204.pin /etc/apt/preferences.d/cuda-repository-pin-600
!wget https://developer.download.nvidia.com/compute/cuda/11.8.0/local_installers/cuda-repo-ubuntu2204-11-8-local_11.8.0-520.61.05-1_amd64.deb
!dpkg -i cuda-repo-ubuntu2204-11-8-local_11.8.0-520.61.05-1_amd64.deb
!cp /var/cuda-repo-ubuntu2204-11-8-local/cuda-*-keyring.gpg /usr/share/keyrings/
!apt-get update
!apt-get -y install cuda-11.8

--2023-12-18 09:51:40--  https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/cuda-ubuntu2204.pin
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.199.20.126
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.199.20.126|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 190 [application/octet-stream]
Saving to: ‘cuda-ubuntu2204.pin’

cuda-ubuntu2204.pin 100%[===================>]     190  --.-KB/s    in 0s      

2023-12-18 09:51:40 (6.08 MB/s) - ‘cuda-ubuntu2204.pin’ saved [190/190]

--2023-12-18 09:51:41--  https://developer.download.nvidia.com/compute/cuda/11.8.0/local_installers/cuda-repo-ubuntu2204-11-8-local_11.8.0-520.61.05-1_amd64.deb
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.199.20.126
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.199.20.126|:443... connected.
HTTP request sent, awaiting response... 200 O

In [37]:
!ls -d /usr/local/cuda-*
!which nvcc

/usr/local/cuda-11  /usr/local/cuda-11.8  /usr/local/cuda-12  /usr/local/cuda-12.2
/usr/local/cuda/bin/nvcc


In [38]:
import os
p = os.getenv('PATH')
ld = os.getenv('LD_LIBRARY_PATH')
os.environ['PATH'] = f"/usr/local/cuda-11.8/bin:{p}"
os.environ['LD_LIBRARY_PATH'] = f"/usr/local/cuda-11.8/lib64:{ld}"
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [39]:
# This get the RAPIDS-Colab install files and test check your GPU.  Run this and the next cell only.
# Please read the output of this cell.  If your Colab Instance is not RAPIDS compatible, it will warn you and give you remediation steps.
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/env-check.py

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 413, done.
remote: Counting objects: 100% (144/144), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 413 (delta 103), reused 58 (delta 54), pack-reused 269
Receiving objects: 100% (413/413), 113.29 KiB | 7.08 MiB/s, done.
Resolving deltas: 100% (205/205), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 2.0 MB/s eta 0:00:00
***********************************************************************
Woo! Your instance has the right kind of GPU, a Tesla T4!
We will now install RAPIDS via pip!  Please stand by, should be quick...
***********************************************************************



In [ ]:
# This will update the Colab environment and restart the kernel.  Don't run the next cell until you see the session crash.
!bash rapidsai-csp-utils/colab/update_gcc.sh
import os
os._exit(00)

Updating your Colab environment.  This will restart your kernel.  Don't Panic!
PPA publishes dbgsym, you may need to include 'main/debug' component
Repository: 'deb https://ppa.launchpadcontent.net/ubuntu-toolchain-r/test/ubuntu/ jammy main'
Description:
Toolchain test builds; see https://wiki.ubuntu.com/ToolChain

More info: https://launchpad.net/~ubuntu-toolchain-r/+archive/ubuntu/test
Adding repository.
Adding deb entry to /etc/apt/sources.list.d/ubuntu-toolchain-r-ubuntu-test-jammy.list
Adding disabled deb-src entry to /etc/apt/sources.list.d/ubuntu-toolchain-r-ubuntu-test-jammy.list
Adding key to /etc/apt/trusted.gpg.d/ubuntu-toolchain-r-ubuntu-test.gpg with fingerprint 60C317803A41BA51845E371A1E9377A2BA9EF27F
Get:1 file:/var/cuda-repo-ubuntu2204-11-8-local  InRelease [1,575 B]
Hit:2 http://security.ubuntu.com/ubuntu jammy-security InRelease
Get:1 file:/var/cuda-repo-ubuntu2204-11-8-local  InRelease [1,575 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://arch

In [1]:
#ref: https://docs.rapids.ai/deployment/stable/platforms/colab/
# !git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

***********************************************************************
Woo! Your instance has the right kind of GPU, a Tesla T4!
We will now install RAPIDS cuDF, cuML, and cuGraph via pip! 
Please stand by, should be quick...
***********************************************************************

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.6/502.6 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 GB 740.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 GB 413.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.3/140.3 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.3/82.3 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 520.0/520.0 MB 3.0 MB/

In [2]:
import cudf
cudf.__version__

'23.10.02'

In [3]:
import cudf
import io, requests

# download CSV file from GitHub
url="https://github.com/plotly/datasets/raw/master/tips.csv"
content = requests.get(url).content.decode('utf-8')

# read CSV from memory
tips_df = cudf.read_csv(io.StringIO(content))
tips_df['tip_percentage'] = tips_df['tip']/tips_df['total_bill']*100

# display average tip by dining party size
print(tips_df.groupby('size').tip_percentage.mean())

size
6    15.622920
1    21.729202
4    14.594901
3    15.215685
2    16.571919
5    14.149549
Name: tip_percentage, dtype: float64


In [ ]:
# !pip install torch==1.11.0+cu113 torchvision==0.12.0+cu113 torchaudio==0.11.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html #pipに対応したverを入れなおす、できるだけ近いVerで

In [4]:
# !pip install torch==1.10.1+cu113 torchvision==0.11.2+cu113 torchaudio==0.10.1+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
# !pip install torchtext==0.11.1 pytorch-lightning==1.5.8 pytorch-metric-learning==1.1.0 timm==0.5.4 augly==0.2.1 addict==2.4.0 rich==11.0.0 faiss-cpu==1.7.2
!pip install torch==2.1.0 torchvision==0.16.0 torchaudio==2.1.0 #cuda11.8
!pip install -U scikit-learn
!apt-get install python3-magic
!pip install pytorch_lightning==1.5.8 pytorch-metric-learning==1.1.0 timm==0.5.4 addict==2.4.0 rich==11.0.0 faiss-cpu==1.7.2 augly==0.2.1 #not insatll augly

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 38.9 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  python3-magic
0 upgraded, 1 newly installed, 0 to remove and 57 not upgraded.
Need to get 12.6 kB of archives.
After this operation, 52.2 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 python3-magic all 2:0.4.24-2 [12.6 kB]
Fetched 12.6 kB in 0s (320 kB/s)
Selecting previously unselected package python3-magic.
(Reading database ... 129089 files and directories currently installed.)
Preparing to unpack .../python3-magic_2%3a0.4.24-2_all.deb ...
Unpacking python3-magic (2:0.4.24-2) ...
Setting up python3-magic (2:0.4.24-2) ...
     ━━━━━━━━━━━

In [5]:
# !pip install pytorch_lightning==1.5.8
#!pip install torchtext==0.11.0 #cannot installed 0.11.1
!pip install torchtext==0.6.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.5 MB/s eta 0:00:00
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.16.0
    Uninstalling torchtext-0.16.0:
      Successfully uninstalled torchtext-0.16.0


In [6]:
import torchtext
torchtext.__version__

'0.6.0'

In [7]:
!pip install pytorch_lightning==1.5.8

## Import my training models

In [8]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 660.5/660.5 kB 7.7 MB/s eta 0:00:00


In [9]:
%cd /content

/content


In [10]:
#!/usr/bin/env python
# coding: utf-8

# # Import

# In[ ]:


import os
import gc
import json
import yaml
import copy
import typing as tp
from pathlib import Path
from tqdm import tqdm

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image
import cv2

import torch
from torch import nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

import pytorch_lightning as pl
from pytorch_lightning import Trainer, LightningDataModule, LightningModule

import albumentations as A
from albumentations.pytorch import ToTensorV2

import timm

from cuml.neighbors import NearestNeighbors

import warnings
warnings.simplefilter('ignore')
from ultralytics import YOLO # yolov8

# # Config

# In[ ]:


IMG_SIZE = 384

SETTINGS = json.load(open('settings.json', 'r'))
if SETTINGS["USE_PRETRAIN"].lower()=='true':
    MODEL_DIR = SETTINGS["PRETRAINED_MODEL_CHECKPOINT_DIR"]
else:
    MODEL_DIR = SETTINGS["MODEL_CHECKPOINT_DIR"]
DEVICE = SETTINGS['SUBMISSION_DEVICE']
METRIC = 'euclidean'

print(f'use model dir = {MODEL_DIR}')


# # Define Function

# ## preprocess

# In[ ]:

YOLO_CROP_BESTPT="/content/my-yolov8-model/best.pt"
CROP_MODEL= YOLO(YOLO_CROP_BESTPT)

def preprocess(path):
  img = cv2.imread(path)
  results = CROP_MODEL.predict(source=img)

  if not results[0].cuda().boxes.xyxy.shape[0]: #もしバウンディングボックスが見つからなかったら、普通に切り抜きされた画像を返す
    return easy_crop_image(path)
  x1 = int(results[0].cuda().boxes.xyxy[0][0].item()) #検出領域(バウンディングボックス)のx座標1
  y1 = int(results[0].cuda().boxes.xyxy[0][1].item()) #検出領域(バウンディングボックス)のy座標1
  x2 = int(results[0].cuda().boxes.xyxy[0][2].item()) #検出領域(バウンディングボックス)のx座標2
  y2 = int(results[0].cuda().boxes.xyxy[0][3].item()) #検出領域(バウンディングボックス)のy座標2
  yolov8_cropped = img[y1:y2, x1:x2]
  #  https://github.com/anyai-28/nishika_jpo_2nd_solution/blob/main/preprocess.py
  if np.min([yolov8_cropped.shape[0],yolov8_cropped.shape[1]]) < 50: # 切り取られた画像の最小の寸法（高さまたは幅）が50ピクセル未満かを確認
    if (yolov8_cropped.shape[0] * yolov8_cropped.shape[1]) < (img.shape[0] * img.shape[1]*0.05): # croppedの面積が元の画像の5%未満か確認
      return easy_crop_image(path)
  return yolov8_cropped

def easy_crop_image(path):
    img = cv2.imread(path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # img2 = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)[1]
    img2 = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY)[1]
    contours = cv2.findContours(img2, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)[0]
    x1 = [] #x座標の最小値
    y1 = [] #y座標の最小値
    x2 = [] #x座標の最大値
    y2 = [] #y座標の最大値
    for i in range(1, len(contours)):
        ret = cv2.boundingRect(contours[i])
        x1.append(ret[0])
        y1.append(ret[1])
        x2.append(ret[0] + ret[2])
        y2.append(ret[1] + ret[3])
    if x1:
        x1_min = min(x1)
        y1_min = min(y1)
        x2_max = max(x2)
        y2_max = max(y2)
        crop_img = img[y1_min:y2_max, x1_min:x2_max]
        # crop後の画像が小さすぎる場合は、cropしない
        if np.min([crop_img.shape[0],crop_img.shape[1]]) < 50:
            if (crop_img.shape[0]*crop_img.shape[1]) < (img.shape[0]*img.shape[1]*0.1):
                crop_img = img
    else:
        crop_img = img
    return crop_img



# ## Transform

# In[ ]:


transform = A.Compose([
    A.LongestMaxSize(p=1.0, max_size=IMG_SIZE),
    A.PadIfNeeded(p=1.0, min_height=IMG_SIZE, min_width=IMG_SIZE, border_mode=0, value=[255, 255, 255]),
    A.Affine(p=1.0, scale=0.9, cval=[255, 255, 255]),
    A.Normalize(p=1.0, mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])


# ## Model

# In[ ]:


class ImageEmbeddingModel(nn.Module):

    def __init__(
        self, base_name: str, fc_dim: int,
        pretrained=False, in_channels: int=3, fc_drop=0.0, use_bn=False, drop_path_rate=0.0
    ):
        """Initialize"""
        self.base_name = base_name
        super(ImageEmbeddingModel, self).__init__()

        # # prepare backbone
        if hasattr(timm.models, base_name):
            base_model = timm.create_model(
                base_name, num_classes=0, pretrained=pretrained, in_chans=in_channels, drop_path_rate=drop_path_rate)
            in_features = base_model.num_features
            print("load imagenet pretrained:", pretrained)
        else:
            raise NotImplementedError

        self.backbone = base_model
        print(f"{base_name}: {in_features}")

        self.fc = nn.Linear(in_features, fc_dim, bias=False)
        self.bn = nn.BatchNorm1d(fc_dim)
        self._init_params()
        self.use_bn = use_bn

    def _init_params(self):
        nn.init.xavier_normal_(self.fc.weight)
        nn.init.constant_(self.bn.weight, 1)
        nn.init.constant_(self.bn.bias, 0)

    def forward(self, x):
        batch_size = x.shape[0]
        x = self.backbone(x)
        x = self.fc(x)
        if self.use_bn:
            x = self.bn(x)
        x = F.normalize(x)
        return x


# In[ ]:


import torch
import torch.nn as nn
import torch.nn.functional as F
from timm.models.layers import trunc_normal_, DropPath
from timm.models.registry import register_model

class Block(nn.Module):
    r""" ConvNeXt Block. There are two equivalent implementations:
    (1) DwConv -> LayerNorm (channels_first) -> 1x1 Conv -> GELU -> 1x1 Conv; all in (N, C, H, W)
    (2) DwConv -> Permute to (N, H, W, C); LayerNorm (channels_last) -> Linear -> GELU -> Linear; Permute back
    We use (2) as we find it slightly faster in PyTorch

    Args:
        dim (int): Number of input channels.
        drop_path (float): Stochastic depth rate. Default: 0.0
        layer_scale_init_value (float): Init value for Layer Scale. Default: 1e-6.
    """
    def __init__(self, dim, drop_path=0., layer_scale_init_value=1e-6):
        super().__init__()
        self.dwconv = nn.Conv2d(dim, dim, kernel_size=7, padding=3, groups=dim) # depthwise conv
        self.norm = LayerNorm(dim, eps=1e-6)
        self.pwconv1 = nn.Linear(dim, 4 * dim) # pointwise/1x1 convs, implemented with linear layers
        self.act = nn.GELU()
        self.pwconv2 = nn.Linear(4 * dim, dim)
        self.gamma = nn.Parameter(layer_scale_init_value * torch.ones((dim)),
                                    requires_grad=True) if layer_scale_init_value > 0 else None
        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()

    def forward(self, x):
        input = x
        x = self.dwconv(x)
        x = x.permute(0, 2, 3, 1) # (N, C, H, W) -> (N, H, W, C)
        x = self.norm(x)
        x = self.pwconv1(x)
        x = self.act(x)
        x = self.pwconv2(x)
        if self.gamma is not None:
            x = self.gamma * x
        x = x.permute(0, 3, 1, 2) # (N, H, W, C) -> (N, C, H, W)

        x = input + self.drop_path(x)
        return x

class ConvNeXt(nn.Module):
    r""" ConvNeXt
        A PyTorch impl of : `A ConvNet for the 2020s`  -
          https://arxiv.org/pdf/2201.03545.pdf
    Args:
        in_chans (int): Number of input image channels. Default: 3
        num_classes (int): Number of classes for classification head. Default: 1000
        depths (tuple(int)): Number of blocks at each stage. Default: [3, 3, 9, 3]
        dims (int): Feature dimension at each stage. Default: [96, 192, 384, 768]
        drop_path_rate (float): Stochastic depth rate. Default: 0.
        layer_scale_init_value (float): Init value for Layer Scale. Default: 1e-6.
        head_init_scale (float): Init scaling value for classifier weights and biases. Default: 1.
    """
    def __init__(self, in_chans=3, num_classes=1000,
                 depths=[3, 3, 9, 3], dims=[96, 192, 384, 768], drop_path_rate=0.,
                 layer_scale_init_value=1e-6, head_init_scale=1.,
                 ):
        super().__init__()
        self.dims = dims

        self.downsample_layers = nn.ModuleList() # stem and 3 intermediate downsampling conv layers
        stem = nn.Sequential(
            nn.Conv2d(in_chans, dims[0], kernel_size=4, stride=4),
            LayerNorm(dims[0], eps=1e-6, data_format="channels_first")
        )
        self.downsample_layers.append(stem)
        for i in range(3):
            downsample_layer = nn.Sequential(
                    LayerNorm(dims[i], eps=1e-6, data_format="channels_first"),
                    nn.Conv2d(dims[i], dims[i+1], kernel_size=2, stride=2),
            )
            self.downsample_layers.append(downsample_layer)

        self.stages = nn.ModuleList() # 4 feature resolution stages, each consisting of multiple residual blocks
        dp_rates=[x.item() for x in torch.linspace(0, drop_path_rate, sum(depths))]
        cur = 0
        for i in range(4):
            stage = nn.Sequential(
                *[Block(dim=dims[i], drop_path=dp_rates[cur + j],
                layer_scale_init_value=layer_scale_init_value) for j in range(depths[i])]
            )
            self.stages.append(stage)
            cur += depths[i]

        self.norm = nn.LayerNorm(dims[-1], eps=1e-6) # final norm layer
        # self.head = nn.Linear(dims[-1], num_classes)
        self.head = nn.Identity()

        self.apply(self._init_weights)
        # self.head.weight.data.mul_(head_init_scale)
        # self.head.bias.data.mul_(head_init_scale)

    def _init_weights(self, m):
        if isinstance(m, (nn.Conv2d, nn.Linear)):
            trunc_normal_(m.weight, std=.02)
            nn.init.constant_(m.bias, 0)

    def forward_features(self, x):
        for i in range(4):
            x = self.downsample_layers[i](x)
            x = self.stages[i](x)
        return self.norm(x.mean([-2, -1])) # global average pooling, (N, C, H, W) -> (N, C)

    def forward(self, x):
        x = self.forward_features(x)
        x = self.head(x)
        return x

class LayerNorm(nn.Module):
    r""" LayerNorm that supports two data formats: channels_last (default) or channels_first.
    The ordering of the dimensions in the inputs. channels_last corresponds to inputs with
    shape (batch_size, height, width, channels) while channels_first corresponds to inputs
    with shape (batch_size, channels, height, width).
    """
    def __init__(self, normalized_shape, eps=1e-6, data_format="channels_last"):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(normalized_shape))
        self.bias = nn.Parameter(torch.zeros(normalized_shape))
        self.eps = eps
        self.data_format = data_format
        if self.data_format not in ["channels_last", "channels_first"]:
            raise NotImplementedError
        self.normalized_shape = (normalized_shape, )

    def forward(self, x):
        if self.data_format == "channels_last":
            return F.layer_norm(x, self.normalized_shape, self.weight, self.bias, self.eps)
        elif self.data_format == "channels_first":
            u = x.mean(1, keepdim=True)
            s = (x - u).pow(2).mean(1, keepdim=True)
            x = (x - u) / torch.sqrt(s + self.eps)
            x = self.weight[:, None, None] * x + self.bias[:, None, None]
            return x


@register_model
def convnext_base(pretrained=False, in_22k=False, img_size=224, **kwargs):
    num_classes = 21841 if in_22k else 1000
    model = ConvNeXt(depths=[3, 3, 27, 3], dims=[128, 256, 512, 1024], num_classes=num_classes,**kwargs)
    if pretrained:
        if img_size==224:
            url = model_urls['convnext_base_22k'] if in_22k else model_urls['convnext_base_1k']
        else:
            url = model_urls['convnext_base_22k_384']
        checkpoint = torch.hub.load_state_dict_from_url(url=url, map_location="cpu", check_hash=True)
        model.load_state_dict(checkpoint["model"], strict=False)
    return model

@register_model
def convnext_large(pretrained=False, in_22k=False, img_size=224, **kwargs):
    model = ConvNeXt(depths=[3, 3, 27, 3], dims=[192, 384, 768, 1536], **kwargs)
    if pretrained:
        url = model_urls['convnext_large_22k'] if in_22k else model_urls['convnext_large_1k']
        checkpoint = torch.hub.load_state_dict_from_url(url=url, map_location="cpu")
        model.load_state_dict(checkpoint["model"])
    return model


# In[ ]:


class ConvNextEmbeddingModel(nn.Module):

    def __init__(
        self, base_name: str, fc_dim: int, in_22k=False, img_size=384,
        pretrained=False, drop_path_rate=0.0, use_bn=False
    ):
        """Initialize"""
        self.base_name = base_name
        super(ConvNextEmbeddingModel, self).__init__()

        # # prepare backbone
        base_model = eval(base_name)(pretrained=pretrained,
                                     drop_path_rate=drop_path_rate,
                                     in_22k=in_22k,
                                     img_size=img_size)
        in_features = base_model.dims[-1]
        self.backbone = base_model
        print(f"{base_name}: {in_features}")

        self.fc = nn.Linear(in_features, fc_dim, bias=False)
        self.bn = nn.BatchNorm1d(fc_dim)
        self._init_params()
        self.use_bn = use_bn

    def _init_params(self):
        nn.init.xavier_normal_(self.fc.weight)
        nn.init.constant_(self.bn.weight, 1)
        nn.init.constant_(self.bn.bias, 0)

    def forward(self, x):
        batch_size = x.shape[0]
        x = self.backbone.forward_features(x)
        x = self.fc(x)
        if self.use_bn:
            x = self.bn(x)
        x = F.normalize(x)
        return x


# In[ ]:


class Model(LightningModule):

    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, x):
        out = self.model(x)
        return out


# ## Get Embedding

# In[ ]:


#def get_image_embeddings(query_image, dic_models, transform):
#    query_image = transform(image=query_image)['image']
#    query_image = query_image.to(DEVICE)
#    query_image = query_image.unsqueeze(0)
#
#    cat_embeds = []
#    for _, models in dic_models.items():
#        embed_list = []
#        for model in models:
#            # model.to(DEVICE)
#            # model.eval()
#            with torch.no_grad():
#                embed = model(query_image)
#                embed_list.append(embed.detach().cpu().numpy())
#        cat_embeds.append(np.mean(embed_list, axis=0))
#        del embed_list
#        gc.collect()
#        torch.cuda.empty_cache()
#
#    del query_image
#    gc.collect()
#    torch.cuda.empty_cache()
#    return np.concatenate(cat_embeds, axis=1)

def get_image_embeddings(query_image, dic_models, transform):
    query_image = transform(image=query_image)['image']
    query_image = query_image.to(DEVICE)
    query_image = query_image.unsqueeze(0)

    embeds = []
    for model in dic_models['convnext']:
        with torch.no_grad():
            embed = model(query_image)
            embeds.append(embed.detach().cpu().numpy())
    return np.mean(embeds, axis=0)


# ## KNN

# In[ ]:


def knn_w_postprocess(query_image_embedding, cite_embeddings, train_embeddings, kn_cite, kn_train, df_train, df_cite):

    _, pred_indexs = kn_cite.kneighbors(np.reshape(query_image_embedding, (1, -1)))

    _, sim_train_index = kn_train.kneighbors(np.reshape(query_image_embedding, (1, -1)))
    sim_train_cite_gid = df_train.at[sim_train_index[0][0], 'cite_gid']

    label = df_train.at[sim_train_index[0][0], 'label']
    sim_train_cite_gids = df_train['cite_gid'][df_train['label']==label].unique()

    num = len(sim_train_cite_gids)

    pred_indexs = pred_indexs[0][:(15-num+1)]
    pred_indexs = np.append(pred_indexs, df_cite['gid'][df_cite['gid'].isin(sim_train_cite_gids)].index) # train縺ｮcite繧定ｿｽ蜉

    tmp_dsts = []
    tmp_idxs = []
    for sim_train_cite_gid in sim_train_cite_gids:
        sim_cite_index = df_cite[df_cite['gid']==sim_train_cite_gid].index[0]

        aug_embedding = cite_embeddings[sim_cite_index] + (query_image_embedding-train_embeddings[sim_train_index[0][0]])
        aug_embedding = np.reshape(aug_embedding, (1, -1))
        aug_embedding /= np.linalg.norm(aug_embedding, axis=1, keepdims=True)
        tmp_dst, tmp_idx = kn_cite.kneighbors(aug_embedding)
        tmp_dsts.extend(tmp_dst[0])
        tmp_idxs.extend(tmp_idx[0])
        pred_indexs = np.append(pred_indexs, tmp_idx[0][0]) # 荳逡ｪ莨ｼ縺ｦ縺・◆id繧定ｿｽ蜉

    sort_idx = np.argsort(tmp_dsts)
    pred_indexs = np.append(pred_indexs, [tmp_idxs[i] for i in sort_idx]) # 鬘樔ｼｼ蠎ｦ縺碁ｫ倥°縺｣縺滄・↓霑ｽ蜉
    pred_indexs = sorted(set(pred_indexs), key=pred_indexs.tolist().index) # 鬆・ｺ上ｒ菫晄戟縺励◆縺ｾ縺ｾ驥崎､・炎髯､
    pred_indexs = pred_indexs[:20]

    return pred_indexs


# ## get_result

# In[ ]:


def get_result(query_image_path, dic_models, transform, cite_embeddings, train_embeddings, kn_cite, kn_train, df_train, df_cite):
    query_image = preprocess(query_image_path)
    #print(query_image)
    #print(type(query_image))
    #exit() # debug
    query_image_embedding = get_image_embeddings(query_image, dic_models, transform)
    result_indexs = knn_w_postprocess(query_image_embedding, cite_embeddings, train_embeddings, kn_cite, kn_train, df_train, df_cite)
    return result_indexs


# # Data Load

# ## Model

# In[ ]:


print('===== Load Models =====')

dic_models = {}
dic_model_paths = {
                    'convnext':[
                        MODEL_DIR + 'convnext/last_fold_0.ckpt',
                        MODEL_DIR + 'convnext/last_fold_1.ckpt',
                        MODEL_DIR + 'convnext/last_fold_2.ckpt',
                        MODEL_DIR + 'convnext/last_fold_3.ckpt',
                        MODEL_DIR + 'convnext/last_fold_4.ckpt'
                    ],
                   #,
                   # 'swin':[
                   #     MODEL_DIR + 'swin/last_fold_0.ckpt',
                   #     MODEL_DIR + 'swin/last_fold_1.ckpt',
                   #     MODEL_DIR + 'swin/last_fold_2.ckpt',
                   #     MODEL_DIR + 'swin/last_fold_3.ckpt',
                   #     MODEL_DIR + 'swin/last_fold_4.ckpt'
                   # ],
                  }

models = []
for path in dic_model_paths['convnext']:
    model = ConvNextEmbeddingModel('convnext_base', 512, img_size=IMG_SIZE, pretrained=False)
    model = Model.load_from_checkpoint(path, model=model, map_location='cpu', strict=False)
    model.to(DEVICE)
    model.eval()
    models.append(model)
dic_models['convnext'] = models

#for k, model_paths in dic_model_paths.items():
#    models = []
#    if k == 'swin':
#        for path in model_paths:
#            model = ImageEmbeddingModel('swin_large_patch4_window12_384', 512, use_bn=True, pretrained=False)
#            model = Model.load_from_checkpoint(path, model=model, map_location='cpu')
#            model.to(DEVICE)
#            model.eval()
#            models.append(model)
#        dic_models['swin'] = models
#    else:
#        for path in model_paths:
#            model = ConvNextEmbeddingModel('convnext_base', 512, img_size=IMG_SIZE, pretrained=False)
#            model = Model.load_from_checkpoint(path, model=model, map_location='cpu', strict=False)
#            model.to(DEVICE)
#            model.eval()
#            models.append(model)
#        dic_models['convnext'] = models
#

# ## Embeddings

# In[ ]:


print('===== Load Embeddings =====')


# In[ ]:
cite_embeddings = np.load(MODEL_DIR + 'convnext/cite_mean_embed.npy')
train_embeddings = np.load(MODEL_DIR + 'convnext/train_mean_embed.npy')

#cite_embeddings = []
#cite_embeddings.append(np.load(MODEL_DIR + 'convnext/cite_mean_embed.npy'))
#cite_embeddings.append(np.load(MODEL_DIR + 'swin/cite_mean_embed.npy'))
#cite_embeddings = np.concatenate(cite_embeddings, axis=1)


# In[ ]:
#train_embeddings = []
#train_embeddings.append(np.load(MODEL_DIR + 'convnext/train_mean_embed.npy'))
#train_embeddings.append(np.load(MODEL_DIR + 'swin/train_mean_embed.npy'))
#train_embeddings = np.concatenate(train_embeddings, axis=1)
#train_embeddings.append(np.load(MODEL_DIR + 'convnext/train_mean_embed.npy'))

# ## KNN

# In[ ]:


print('===== Set KNN =====')

kn_cite = NearestNeighbors(n_neighbors=20, metric=METRIC)
kn_train = NearestNeighbors(n_neighbors=20, metric=METRIC)

kn_cite.fit(cite_embeddings)
kn_train.fit(train_embeddings)


# ## DataFrame

# In[ ]:


#df_train = pd.read_csv(SETTINGS['RAW_DATA_DIR'] + 'train.csv')
df_train = pd.read_csv(SETTINGS['PROCESSED_DATA_DIR'] + 'train_w_group_label.csv')
df_cite = pd.read_csv(SETTINGS['RAW_DATA_DIR'] + 'cite.csv')
df_test = pd.read_csv(SETTINGS['RAW_DATA_DIR'] + 'test.csv')


# In[ ]:


#dic_id2label = {}
#for i, (r_id, c_id) in enumerate(zip(df_train['gid'].values, df_train['cite_gid'].values)):
#    label = None
#    try:
#        label = dic_id2label[r_id]
#    except:
#        pass
#    try:
#        label = dic_id2label[c_id]
#    except:
#        pass
#
#    if label:
#        dic_id2label[r_id] = label
#        dic_id2label[c_id] = label
#    else:
#        dic_id2label[r_id] = i
#        dic_id2label[c_id] = i
#

# In[ ]:


#df_train['label'] = df_train['gid']
#df_train['label'] = df_train['label'].map(dic_id2label)


# In[ ]:


cite_idx2id = dict(zip(df_cite.index, df_cite['gid']))


# # Make Sub

# In[ ]:


print('===== Make Submit =====')
pred = []
for p in tqdm(df_test['path'].values):
    query_image_path = SETTINGS['RAW_DATA_DIR'] + f"apply_images/{p}"
    #print(query_image_path)
    result_indexs = get_result(query_image_path, dic_models, transform, cite_embeddings, train_embeddings, kn_cite, kn_train, df_train, df_cite)
    result = ' '.join([str(cite_idx2id[i]) for i in result_indexs])
    pred.append(result)


# In[ ]:


smpl_sub = pd.read_csv(SETTINGS['RAW_DATA_DIR'] + "sample_submission.csv")
sub_df = smpl_sub.copy()
sub_df['cite_gid'] = pred
sub_df.head()


# In[ ]:


os.makedirs(SETTINGS['SUBMISSION_DIR'], exist_ok=True)
sub_df.to_csv(SETTINGS['SUBMISSION_DIR'] + 'submit.csv', index=False)


# In[ ]:


print('===== Finish =====')



use model dir = ./models/
===== Load Models =====
convnext_base: 1024
convnext_base: 1024
convnext_base: 1024
convnext_base: 1024
convnext_base: 1024
===== Load Embeddings =====
===== Set KNN =====
===== Make Submit =====


  0%|          | 0/1542 [00:00<?, ?it/s]


WARNING ⚠️ NMS time limit 0.550s exceeded
0: 448x640 2 trademarks, 163.5ms
Speed: 24.3ms preprocess, 163.5ms inference, 1487.5ms postprocess per image at shape (1, 3, 448, 640)


  0%|          | 1/1542 [00:06<2:44:09,  6.39s/it]


0: 384x640 1 trademark, 76.8ms
Speed: 3.4ms preprocess, 76.8ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)


...


0: 384x640 1 trademark, 23.5ms
Speed: 3.9ms preprocess, 23.5ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)


100%|█████████▉| 1541/1542 [11:24<00:00,  2.02it/s]


0: 512x640 1 trademark, 28.7ms
Speed: 5.6ms preprocess, 28.7ms inference, 2.3ms postprocess per image at shape (1, 3, 512, 640)


100%|██████████| 1542/1542 [11:24<00:00,  2.25it/s]

===== Finish =====


In [11]:
%cd /content/submissions
from google.colab import files
files.download('submit.csv')

/content/submissions


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>